# Deploy sklearn model to webservice endpoint

description: deploy sklearn ridge model trained on diabetes data to AKS

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# azure ml settings
experiment_name = "sklearn-diabetes-mlproject-example"

In [ ]:
import mlflow

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

In [ ]:
model = None
runs = ws.experiments[experiment_name].get_runs()
run = next(runs)
while run.get_status() != "Completed" or model is None:
    run = next(runs)
    try:
        model = run.register_model(experiment_name, model_path="model")
    except:
        pass

run

In [ ]:
from azureml.core.webservice import AksWebservice

aks_config = AksWebservice.deploy_configuration(
    compute_target_name="aks-cpu-deploy",
    cpu_cores=2,
    memory_gb=5,
    tags={"data": "diabetes", "method": "sklearn"},
    description="Predict using webservice",
)

In [ ]:
import mlflow.azureml
from random import randint

webservice, azure_model = mlflow.azureml.deploy(
    model_uri=f"runs:/{run.id}/model",
    workspace=ws,
    deployment_config=aks_config,
    service_name="sklearn-diabetes-" + str(randint(10000, 99999)),
    model_name="sklearn-diabetes-example",
)

In [ ]:
import pandas as pd

data = pd.read_csv(prefix.joinpath("data", "raw", "diabetes", "diabetes.csv"))

sample = data.drop(["progression"], axis=1).iloc[[0]]

query_input = sample.to_json(orient="split")
query_input = eval(query_input)
query_input.pop("index", None)

In [ ]:
# if (key) auth is enabled, retrieve the API keys. AML generates two keys.
key1, Key2 = webservice.get_keys()
print(key1)

# # if token auth is enabled, retrieve the token.
# access_token, refresh_after = webservice.get_token()

In [ ]:
# If (key) auth is enabled, don't forget to add key to the HTTP header.
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + key1,
}

# # If token auth is enabled, don't forget to add token to the HTTP header.
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

In [ ]:
import requests
import json

response = requests.post(
    url=webservice.scoring_uri, data=json.dumps(query_input), headers=headers
)
print(response.text)

In [ ]:
webservice.delete()